In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Twente/Q2/RPCN---November-2025

/content/drive/MyDrive/Twente/Q2/RPCN---November-2025


In [ ]:
#!/usr/bin/env python3
import os
import pandas as pd
import numpy as np

def calibrate_imu_series(directory: str, g: float = 9.80665):
    """
    Calibrate an IMU from 18 CSV files (space-separated, headers as given).

    Files must be named like  MT_001-000.csv , MT_002-000.csv , ... MT_018-000.csv
    (the numeric part is used for ordering).

    Returns a dict with per-series and overall results.
    """
    # ------------------------------------------------------------
    # 1. List & sort files
    # ------------------------------------------------------------
    csv_files = [f for f in os.listdir(directory) if f.lower().endswith('.csv')]
    if len(csv_files) != 18:
        raise ValueError(f"Expected 18 CSV files, found {len(csv_files)} in {directory}")

    # sort by the number in the filename (001 … 018)
    csv_files.sort(key=lambda x: int(x.split('_')[-1].split('-')[0]))

    # ------------------------------------------------------------
    # 2. Helper: average of a column in one file
    # ------------------------------------------------------------
    def avg_column(filepath: str, col: str) -> float:
        df = pd.read_csv(filepath, sep=r'\s+', usecols=[col])
        return float(df[col].mean())

    # ------------------------------------------------------------
    # 3. Define which files belong to which orientation
    # ------------------------------------------------------------
    # 0-based indices for the three series (0-5, 6-11, 12-17)
    series_idxs = [(0, 6, 12), (1, 7, 13), (2, 8, 14), (3, 9, 15), (4, 10, 16), (5, 11, 17)]

    # Axis → (up indices, down indices, column name)
    axis_cfg = {
        'Z': ( (0, 6, 12),  (1, 7, 13), 'Acc_Z' ),   # 001-down, 002-up
        'X': ( (3, 9, 15),  (2, 8, 14), 'Acc_Y' ),   # 003-down, 004-up
        'Y': ( (5, 11, 17), (4, 10, 16), 'Acc_X' ) # 006-down, 005-up
    }

    # ------------------------------------------------------------
    # 4. Compute per-series values
    # ------------------------------------------------------------
    results = {'Z': [], 'X': [], 'Y': []}

    for axis, (up_idx, down_idx, col) in axis_cfg.items():
        for ser in range(3):                     # three series
            # files for this series, this orientation
            up_files   = [os.path.join(directory, csv_files[i]) for i in up_idx[ser:ser+1]]
            down_files = [os.path.join(directory, csv_files[i]) for i in down_idx[ser:ser+1]]

            f_up   = avg_column(up_files[0], col)
            f_down = avg_column(down_files[0], col)

            b = (f_up + f_down) / 2.0
            S = (f_up - f_down - 2 * g) / (2 * g)

            results[axis].append({'series': ser + 1, 'f_up': f_up, 'f_down': f_down,
                                  'b': b, 'S': S})

    # ------------------------------------------------------------
    # 5. Overall average
    # ------------------------------------------------------------
    overall = {}
    for axis in results:
        b_vals = [r['b'] for r in results[axis]]
        S_vals = [r['S'] for r in results[axis]]
        overall[axis] = {'b_avg': np.mean(b_vals), 'S_avg': np.mean(S_vals)}

    # ------------------------------------------------------------
    # 6. Pretty printing
    # ------------------------------------------------------------
    def print_series(axis: str):
        print(f"\n=== {axis}-axis calibration ===")
        print(f"{'Series':>6}  {'f_up':>10}  {'f_down':>10}  {'b':>12}  {'S':>12}")
        print("-" * 52)
        for r in results[axis]:
            print(f"{r['series']:6d}  {r['f_up']:10.6f}  {r['f_down']:10.6f}  "
                  f"{r['b']:12.6f}  {r['S']:12.6f}")
        print("-" * 52)
        print(f"{'AVG':>6}  {'':>10}  {'':>10}  {overall[axis]['b_avg']:12.6f}  {overall[axis]['S_avg']:12.6f}")

    print_series('Z')
    print_series('X')
    print_series('Y')

    # ------------------------------------------------------------
    # 7. Return structured data (optional)
    # ------------------------------------------------------------
    return {'per_series': results, 'overall': overall}


result = calibrate_imu_series('/content/drive/MyDrive/Twente/Q2/RPCN---November-2025/Assignment 1/Data/csv')



=== Z-axis calibration ===
Series        f_up      f_down             b             S
----------------------------------------------------
     1    9.806066   -9.818758     -0.006346      0.000588
     2    9.806664   -9.816788     -0.005062      0.000518
     3    9.803234   -9.822028     -0.009397      0.000610
----------------------------------------------------
   AVG                             -0.006935      0.000572

=== X-axis calibration ===
Series        f_up      f_down             b             S
----------------------------------------------------
     1    9.868271   -9.745906      0.061183      0.000045
     2    9.869006   -9.747658      0.060674      0.000172
     3    9.867701   -9.747367      0.060167      0.000090
----------------------------------------------------
   AVG                              0.060675      0.000102

=== Y-axis calibration ===
Series        f_up      f_down             b             S
----------------------------------------------------
  

In [ ]:
#!/usr/bin/env python3
import os
import pandas as pd
import numpy as np

def calibrate_gyro_series(directory: str):
    """
    Calibrate Gyroscope from 18 CSV files (space-separated).

    Assumes files are named with numeric ordering (e.g., MT_001-000.csv ... MT_018-000.csv).
    Logic:
      - Pairs of files correspond to opposite orientations for a specific axis.
      - The 'Nominal Z' (Vertical) axis is identified by the largest accelerometer reading.
      - 'Up' is defined as the orientation where the vertical axis acceleration is positive.
      - 'Down' is defined as the orientation where the vertical axis acceleration is negative.
      - Calibration reference is the vertical component of Earth's rotation at Enschede.
    """

    # ------------------------------------------------------------
    # 1. Constants for Enschede, Netherlands
    # ------------------------------------------------------------
    # Standard gravity (used only if checking acc magnitude, though sign is enough)
    g = 9.80665

    # Earth's rotation rate (approx 15 deg/hour)
    OMEGA_E = 7.292115e-5  # rad/s

    # Enschede Latitude (~52.22 degrees North)
    LAT_ENSCHEDE = 52.23767622080391

    # Vertical component of Earth's rotation: Omega_up = Omega_E * sin(latitude)
    # This is the input rate sensed by the vertical axis.
    OMEGA_V = OMEGA_E * np.sin(np.radians(LAT_ENSCHEDE))

    print(f"Constants:")
    print(f"  Earth Rate (Omega_E): {OMEGA_E:.6e} rad/s")
    print(f"  Enschede Lat (phi)  : {LAT_ENSCHEDE:.4f} degrees")
    print(f"  Vertical Rate (inp) : {OMEGA_V:.6e} rad/s\n")

    # ------------------------------------------------------------
    # 2. List & sort files
    # ------------------------------------------------------------
    csv_files = [f for f in os.listdir(directory) if f.lower().endswith('.csv')]
    if len(csv_files) != 18:
        raise ValueError(f"Expected 18 CSV files, found {len(csv_files)} in {directory}")

    # Sort by the number in the filename (001 … 018)
    csv_files.sort(key=lambda x: int(x.split('_')[-1].split('-')[0]))

    # ------------------------------------------------------------
    # 3. Helper: Get means of all columns
    # ------------------------------------------------------------
    def get_means(filepath: str) -> pd.Series:
        # Reads space-separated (or tab-separated) CSV
        # Headers assumed: PacketCounter SampleTimeFine Acc_X Acc_Y Acc_Z Gyr_X Gyr_Y Gyr_Z ...
        df = pd.read_csv(filepath, sep=r'\s+')
        return df.mean(numeric_only=True)

    # ------------------------------------------------------------
    # 4. Define file groupings (Pairs)
    # ------------------------------------------------------------
    # series_idxs maps the file indices for each orientation across the 3 repeated series.
    # Tuple 0: Indices for Orientation 1 (Files 0, 6, 12)
    # Tuple 1: Indices for Orientation 2 (Files 1, 7, 13)
    # ...
    # Pairs are formed by (Orientation 1, Orientation 2), (Orientation 3, Orientation 4), etc.
    series_idxs = [(0, 6, 12), (1, 7, 13), (2, 8, 14), (3, 9, 15), (4, 10, 16), (5, 11, 17)]

    # The pairs corresponding to opposite faces are indices (0,1), (2,3), (4,5) in the series_idxs list.
    pair_group_indices = [(0, 1), (2, 3), (4, 5)]

    # Store results per axis
    results = {'X': [], 'Y': [], 'Z': []}

    # ------------------------------------------------------------
    # 5. Process each series and pair
    # ------------------------------------------------------------
    # Iterate over the 3 series repetitions (k=0, 1, 2)
    for k in range(3):
        for (group_idx_a, group_idx_b) in pair_group_indices:

            # Get actual file indices for this specific series iteration
            file_idx_A = series_idxs[group_idx_a][k]
            file_idx_B = series_idxs[group_idx_b][k]

            path_A = os.path.join(directory, csv_files[file_idx_A])
            path_B = os.path.join(directory, csv_files[file_idx_B])

            # Calculate means
            means_A = get_means(path_A)
            means_B = get_means(path_B)

            # --- Identify Nominal (Vertical) Axis ---
            # We look at the accelerometer data in the first file (A) of the pair.
            # The axis with the largest absolute acceleration is the vertical one.
            acc_cols = ['Acc_X', 'Acc_Y', 'Acc_Z']
            acc_vals_A = means_A[acc_cols]

            # Find column name with max absolute value (e.g., 'Acc_Y')
            nominal_acc_col = acc_vals_A.abs().idxmax()
            axis_name = nominal_acc_col.split('_')[1] # Extract 'X', 'Y', or 'Z'

            # --- Determine Up vs Down ---
            # "Up is equal to positive sign alongside the nominal-z"
            # "Down is negative sign"
            acc_val_A = means_A[nominal_acc_col]

            # Get Gyroscope readings for the SAME axis
            gyro_col = f'Gyr_{axis_name}'
            gyr_val_A = means_A[gyro_col]
            gyr_val_B = means_B[gyro_col]

            if acc_val_A > 0:
                # A is Up, B is Down
                f_up = gyr_val_A
                f_down = gyr_val_B
            else:
                # A is Down, B is Up
                f_up = gyr_val_B
                f_down = gyr_val_A

            # --- Calculate Bias and Scale ---
            # Measurement Model: y = (1 + S) * u + b
            # Up input:   u_up   = +OMEGA_V
            # Down input: u_down = -OMEGA_V
            #
            # y_up   = (1 + S) * OMEGA_V + b
            # y_down = (1 + S) * (-OMEGA_V) + b
            #
            # Sum:  y_up + y_down = 2*b  -> b = (y_up + y_down) / 2
            # Diff: y_up - y_down = (1 + S) * 2 * OMEGA_V
            #       S = (y_up - y_down) / (2 * OMEGA_V) - 1
            #       S = (y_up - y_down - 2 * OMEGA_V) / (2 * OMEGA_V)

            b = (f_up + f_down) / 2.0

            meas_diff = f_up - f_down
            ref_diff  = 2.0 * OMEGA_V

            S = (meas_diff - ref_diff) / ref_diff

            results[axis_name].append({
                'series': k + 1,
                'f_up': f_up,
                'f_down': f_down,
                'b': b,
                'S': S
            })

    # ------------------------------------------------------------
    # 6. Compute Overall Averages
    # ------------------------------------------------------------
    overall = {}
    for axis in ['X', 'Y', 'Z']:
        if results[axis]:
            b_vals = [r['b'] for r in results[axis]]
            S_vals = [r['S'] for r in results[axis]]
            overall[axis] = {'b_avg': np.mean(b_vals), 'S_avg': np.mean(S_vals)}
        else:
            overall[axis] = {'b_avg': float('nan'), 'S_avg': float('nan')}

    # ------------------------------------------------------------
    # 7. Print Results
    # ------------------------------------------------------------
    def print_series_results(axis: str):
        print(f"\n=== Gyroscope {axis}-axis calibration ===")
        # f_up/down are in rad/s. b is bias (rad/s). S is scale factor error (unitless).
        print(f"{'Series':>6}  {'f_up (rad/s)':>14}  {'f_down (rad/s)':>14}  {'bias':>12}  {'Scale Err':>12}")
        print("-" * 66)
        for r in results[axis]:
            print(f"{r['series']:6d}  {r['f_up']:14.6e}  {r['f_down']:14.6e}  "
                  f"{r['b']:12.6e}  {r['S']:12.6e}")
        print("-" * 66)
        print(f"{'AVG':>6}  {'':>14}  {'':>14}  {overall[axis]['b_avg']:12.6e}  {overall[axis]['S_avg']:12.6e}")

    print_series_results('Z')
    print_series_results('X')
    print_series_results('Y')

    return {'per_series': results, 'overall': overall}

# Usage Example:
result = calibrate_gyro_series('/content/drive/MyDrive/Twente/Q2/RPCN---November-2025/Assignment 1/Data/csv')

Constants:
  Earth Rate (Omega_E): 7.292115e-05 rad/s
  Enschede Lat (phi)  : 52.2377 degrees
  Vertical Rate (inp) : 5.764839e-05 rad/s


=== Gyroscope Z-axis calibration ===
Series    f_up (rad/s)  f_down (rad/s)          bias     Scale Err
------------------------------------------------------------------
     1   -2.212495e-03   -2.251855e-03  -2.232175e-03  -6.586161e-01
     2   -2.249961e-03   -2.335373e-03  -2.292667e-03  -2.591993e-01
     3   -2.447524e-03   -2.413614e-03  -2.430569e-03  -1.294108e+00
------------------------------------------------------------------
   AVG                                  -2.318470e-03  -7.373079e-01

=== Gyroscope X-axis calibration ===
Series    f_up (rad/s)  f_down (rad/s)          bias     Scale Err
------------------------------------------------------------------
     1   -5.705125e-03   -5.798285e-03  -5.751705e-03  -1.919949e-01
     2   -5.749253e-03   -5.868642e-03  -5.808948e-03  3.548987e-02
     3   -5.878758e-03   -6.092253e-03